# 1 Exercise 2

Description

In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

from typing import Tuple, List

In [2]:
def optimal_percentage(price: float, cost: float, holding_cost: float) -> float:
    return (price - cost)/(price + holding_cost)

In [3]:
def calculate_known_optimal_quantity_normal(cost: float, price: float, holding_cost: float, location: float, scale: float) -> float:
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    return sts.norm.ppf(optimal_percent, loc=location, scale=scale)

In [4]:
def root_mean_squared_error(quantity_estimation_vector: np.ndarray, known_optimal_quantity: float) -> float:
    estimation_error: np.ndarray = quantity_estimation_vector - known_optimal_quantity
    squared_error: float = np.square(estimation_error).sum()

    vector_size: int = quantity_estimation_vector.size
    mean_squared_error: float = squared_error/vector_size
    return np.sqrt(mean_squared_error)


In [5]:
def profit_loss_ratio(profit_estimation_vecor: np.ndarray, known_maximum_profit: float) -> float:
    ratio: np.ndarray = (known_maximum_profit - profit_estimation_vecor)/known_maximum_profit
    vector_size: int = profit_estimation_vecor.size
    return np.absolute(ratio).sum()/vector_size


In [6]:
def normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    mean: float = data_vector.mean()
    var: float = data_vector.var()
    return (mean, np.sqrt(var))

def parametric_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float =  sts.norm.ppf(optimal_percent, loc=mean, scale=var)

    return inv_cdf_quantity

In [7]:
def log_normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    # Dit moet nog even gecheckt worden, ik heb geen idee of dit is hoe je het berekent. Micha
    mean: float = np.log(data_vector).mean()
    var: float = np.log(data_vector).var()
    return (mean, var)

def parametric_log_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float = sts.lognorm.ppf(optimal_percent, loc=np.exp(mean), scale=np.exp(var))

    return inv_cdf_quantity

In [8]:
def non_parametric_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    sorted_data_vector: np.ndarray = data_vector.sort()
    vector_size = data_vector.size
    optimal_percent = optimal_percentage(price, cost, holding_cost)
    optimal_arg_value = int(np.ceil(optimal_percent * vector_size))

    return data_vector[optimal_arg_value - 1]

In [9]:
def normal_monte_carlo_simulation_and_paramatric_estimation(number, target_surface, location, scale, price):
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return parametric_normal_optimal_quantity(1 - target_surface, 0, price, data_vector)


In [10]:
def normal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price):
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return non_parametric_optimal_quantity(1 - target_surface, 0, price, data_vector)

In [11]:
def calculate_normal_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float) -> Tuple[float, float]:
    estimation_results: np.ndarray = np.empty(estimations)
    for i in range(estimations):
        estimation_results[i] = normal_monte_carlo_simulation_and_paramatric_estimation(number, target_surface, location, scale, price)

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    rsme: float = root_mean_squared_error(estimation_results, known_optimal_quantity)
    # Calculate PLR
    plr: float = 0
    return (rsme, plr)
    

In [12]:
def calculate_normal_non_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float):
    estimation_results: np.ndarray = np.empty(estimations)

    for i in range(estimations):
        estimation_results[i] = normal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price)

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    rsme: float = root_mean_squared_error(estimation_results, known_optimal_quantity)
    plr: float = 0 # Calculate
    return (rsme, plr)

In [13]:
def normal():
    locations: List[int] = [50, 100, 200, 500]
    scales: List[int] = [1, 5, 10, 20]
    estimations: int = 1000
    number: List[int] = [10, 50, 100, 200]
    target_surface: List[int] = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    price: int = 1

    for location in locations:
        for scale in scales:
            for n in number:
                for t in target_surface:
                    print(f"n: {n}, t:{t}, loc:{location}, scale: {scale}")
                    parametric_metrics = calculate_normal_parametric_metrics(n, t, scale, price, estimations, location)
                    nonparametric_metrics = calculate_normal_non_parametric_metrics(n, t, scale, price, estimations, location)
                    print(f"parametric rsme: {parametric_metrics[0]}, plr: {parametric_metrics[1]}")
                    print(f"nonparametric rsme: {nonparametric_metrics[0]}, plr {nonparametric_metrics[1]}")
                    print("")